# Computing Thermal Flux

Computing thermal flux is pretty much the same deal as the reflected light. There are just a couple tweaks we will have to make in order to make sure we have the full set of info. 

In [2]:
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi

#plotting
jpi.output_notebook()

Loading BokehJS ...

We will use new Hot Jupiter template to guide us through the exercise. 

In [3]:
opa = jdi.opannection(wave_range=[3,5])

case1 = jdi.inputs()

case1.phase_angle(0) 


#here we are going to have to specify gravity through R and M since we need it in the Flux calc
case1.gravity(mass=1, mass_unit=jdi.u.Unit('M_jup'), 
              radius=1.2, radius_unit=jdi.u.Unit('R_jup'))

#here we are going to have to specify R as well
case1.star(opa, 4000,0.0122,4.437,radius=0.7, radius_unit = jdi.u.Unit('R_sun') )

#atmo
case1.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)
case1.clouds(filename = jdi.HJ_cld(), delim_whitespace=True)

## Return ``PICASO`` Full Ouput

In [4]:
df= case1.spectrum(opa, full_output=True,calculation='thermal') #note the new last key 

wno, fpfs , fp = df['wavenumber'] , df['fpfs_thermal'], df['thermal']
wno, fpfs = jdi.mean_regrid(wno, fpfs, R=150)
wno, fp = jdi.mean_regrid(wno, fpfs, R=150)
full_output = df['full_output']


/Users/nbatalh1/Documents/codes/PICASO/picaso/picaso/atmsetup.py:190: UserWarning: Ignoring x in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)
/Users/nbatalh1/Documents/codes/PICASO/picaso/picaso/atmsetup.py:190: UserWarning: Ignoring KZZ in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)


Here we are getting a few error messages that our example `HJ_cld` had some unrecognized molecules. Looking them over, this looks okay! `x` is not a molecule, and neither is `Kzz`. The code has accurately identified non-molecules in our input.

## Analyzing Thermal Emission Output

All the functionality that we used for reflected light we will also be able to use for thermal emission

### Mixing Ratios and Pressure Temperature plots

In [5]:
jpi.show(jpi.row(
    jpi.mixing_ratio(full_output, plot_height=500), 
    jpi.pt(full_output, plot_height=500)))
#can also input any key word argument acceptable for bokeh.figure: 
#show(jpi.mixing_ratio(full_output, plot_width=500, y_axis_type='linear',y_range=[10,1e-3])) 

## Standard Relative Flux Fp/Fs

The same function as reflected light will work with any of the thermal emission output

In [6]:
jpi.show(jpi.spectrum(wno,fpfs*1e6,plot_width=500,y_axis_type='log'))

## Exploring Thermal Emission Spectrum

This is a useful plot to see the interplay between the computed flux and blackbodies of temperatures at various pressures along the PT profile. 

When you specify a pressure here, it will find the corresponding temperature along the PT profile and plot that blackbody. 

In [7]:
jpi.show(jpi.flux_at_top(df, pressures=[1e-4, 1e-3, 1e-2], R=150,  plot_width=500, plot_height=400))